Internet Resources:

[handson-ml/06_decision_trees.ipynb](https://github.com/ageron/handson-ml/blob/master/06_decision_trees.ipynb)  
[Sefik Ilkin Serengil - A Step by Step CART Decision Tree Example](https://sefiks.com/2018/08/27/a-step-by-step-cart-decision-tree-example/)  
[Google Developers - Tree Classifier from Scratch](https://www.youtube.com/watch?v=LDRbO9a6XPU&t=1s)  
[Victor Zhou - A Simple Explanation of Gini Impurity](https://victorzhou.com/blog/gini-impurity/)


Literature:  

Aurelien geron hands on machine learning page 173

In [1]:
%matplotlib inline
from collections import Counter
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import numpy as np

np.random.seed(42)

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

df = pd.read_csv("data/iris.csv").drop(["sepal_width", "sepal_length"], 1) # data from https://www.kaggle.com/arshid/iris-flower-dataset

label_mappings = {label_str:i for i,label_str in enumerate(df["species"].unique())}
df.replace({"species":label_mappings}, inplace=True)

permutation = np.random.permutation(df.index)
X = np.array(df.drop(["species"], 1))[permutation]
y = np.array(df["species"])[permutation]

df.head()

,petal_length,petal_width,species
0,1.4,0.2,0
1,1.4,0.2,0
2,1.3,0.2,0
3,1.5,0.2,0
4,1.4,0.2,0


In [2]:
# distributions is a list where each element represents the amount of Samples belonging to each classe
# example [20, 30] -> 20 x Sample belonging to class 0 - 30 x Sample belonging to class 1
def gini(distributions): 
    if sum(distributions) == 0: # if there are no samples for the node
        return 0
    num_samples = sum(distributions)
    impurity = 1
    for i in distributions:
        impurity -= (i/num_samples)**2
    return impurity
 

# decision function of the node
def ask(descision_value, ask_value):
    if isinstance(ask_value, int) or isinstance(ask_value, float):    
        return ask_value >= descision_value
    else:
        return ask_value == descision_value


    
    
# end of branch
class TreeNodeEnd():
    def __init__(self, labels, distribution, depth):
        self.label = int(np.bincount(labels).argmax()) # most frequent label in labels
        self.distribution = distribution
        self.depth = depth
        self.gini_score = gini(self.distribution)
        
    def predict(self, value):
        return self.label
    
    def print_tree(self):
        print("\t"*self.depth, "Sample is of class {}".format(self.label))
    
    
    
    
# decision node
class TreeNode():
    def __init__(self, data, labels, distrb, gini, descision_value, feature, depth):
        self.data = data
        self.labels = labels
        self.distribution = distrb
        self.gini_score = gini
        self.feature = feature
        self.descision_value = descision_value
        self.depth = depth
        self.child = {True:None, False:None}
    
    
    def predict(self, value):            
        return self.child[ask(self.descision_value, value[self.feature])].predict(value)
        
    
    def print_tree(self):
        cond = "is greater or equal to" if isinstance(self.descision_value, int) or isinstance(self.descision_value, float) else "is equal to" 
        print("\t"*self.depth, "if feature {} {} {}:".format(self.feature, cond, self.descision_value))
        self.child[True].print_tree()
        print("\t"*self.depth, "else:")
        self.child[False].print_tree()
   

        
    
        
class DescisionTreeClassifier:
    def __init__(self, max_depth = None):
        self.max_depth = max_depth
    
    
    # find the value that produces the lowest gini score and partition the data according to it
    def find_lowest_gini(self, data, labels):
        unique_column_values = {i:list(np.unique(column)) for i,column in enumerate(data.T) } # get only unique feature values
        gini_score = 1
        
        for column in unique_column_values:
            for unique_value in unique_column_values[column]:
                # split data into two groups: True and False
                split_data = {True:[], False:[]}
                split_labels = {True:[], False:[]}
                distrb = {True:[0] * self.class_count, False:[0] * self.class_count}
                for i in range(len(data)):
                    branch = ask(unique_value, data.T[column][i])
                    split_data[branch].append(data[i])
                    split_labels[branch].append(labels[i])
                    distrb[branch][labels[i]] += 1
                
                # gini score is the weighted sum of both branches
                new_gini = (sum(distrb[True]) / len(data) * gini(distrb[True])) + (sum(distrb[False]) / len(data) * gini(distrb[False]))
                
                split_data = {x:np.array(split_data[x]) for x in split_data}
                split_labels = {x:np.array(split_labels[x]) for x in split_labels}
                
                if new_gini < gini_score:
                    gini_score = new_gini
                    lowest_gini_data = split_data, split_labels, distrb, gini_score, unique_value, column
       
        return lowest_gini_data
            
    
    
    
    # recursive function, builds the tree
    def createNextNode(self, data, labels, distribution, depth=0):
        label = int(np.bincount(labels).argmax()) # most often occouring label
        
        # partition data
        split_data, split_labels, distrb, gini_score, descision_value, feature = self.find_lowest_gini(data, labels)
        
        # data has to splitable
        if list(distrb[True]) != distribution and list(distrb[False]) != distribution:
            node = TreeNode(data, labels, distribution, gini_score, descision_value, feature, depth)
            
            for branch in [True, False]:
                depth_reached = False if self.max_depth is None else True if depth+1 == self.max_depth else False
                node_pure = distrb[branch].count(0) >= self.class_count-1
                if depth_reached or node_pure:
                    node.child[branch] = TreeNodeEnd(split_labels[branch], distrb[False], depth+1)
                else:
                    node.child[branch] = self.createNextNode(split_data[branch], split_labels[branch], distrb[branch], depth+1)
            
        else:
            node = TreeNodeEnd(labels, distribution, depth)
        return node
        
    
    def fit(self, data, labels):
        self.class_count = len(np.unique(labels))
        distribution = [0] * len(np.unique(labels))
        for i in range(len(labels)):
            distribution[labels[i]] += 1
        self.root = self.createNextNode(data, labels, distribution)
        
    def predict(self, sample):
        # recoursive function, predicts class of sample
        return self.root.predict(sample)
    
    def print_tree(self):
        # recoursive function, prints decision questions of the tree
        self.root.print_tree()
    
    
    
clf = DescisionTreeClassifier(max_depth=None)
clf.fit(X, y)
clf.print_tree()
clf.predict([1,2])

 if feature 0 is greater or equal to 3.0:
	 if feature 1 is greater or equal to 1.8:
		 if feature 0 is greater or equal to 4.9:
			 Sample is of class 2
		 else:
			 Sample is of class 2
	 else:
		 if feature 0 is greater or equal to 5.0:
			 if feature 1 is greater or equal to 1.6:
				 if feature 0 is greater or equal to 5.8:
					 Sample is of class 2
				 else:
					 Sample is of class 1
			 else:
				 Sample is of class 2
		 else:
			 if feature 1 is greater or equal to 1.7:
				 Sample is of class 2
			 else:
				 Sample is of class 1
 else:
	 Sample is of class 0


0